In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
from typing import Literal, List, Tuple
import matplotlib.pyplot as plt
from collections import defaultdict
import logging
import seaborn as sns

import IPython
import numpy as np
import pandas as pd
from IPython.display import display

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))


import src.utils.custom_log as custom_log
from src.utils.Csv import Csv
from src.utils.set_rcparams import set_rcparams

os.chdir(project_dir)
LOG = logging.getLogger(__name__)
custom_log.init_logger(log_lvl=logging.INFO)
LOG.info("Start in %s", Path().cwd())
set_rcparams()
B_PATH: Path = Path("/mnt") / "q" / "Val_Chain_Sims" / "AB_Testing"
LOG.info("B_PATH: %s", B_PATH)
B_PATH.is_dir()

In [ ]:
def get_data() -> Tuple[pd.DataFrame, List[str]]:
    db = defaultdict(list)
    cases = set([])
    for file in B_PATH.rglob("mes0000"):
        if not file.parent.parent.stem.startswith("Assemblies"):
            continue
        LOG.info("Read: %s", file)
        # read file
        with open(file) as f:
            c = f.readlines()

        # get from file end
        for line in reversed(c):
            if line.startswith(" Elapsed time"):
                parts = line.split()
                db["Time [min]"].append(float(parts[2]) / 60)
                db["Procs"].append({94: 96, 96: 96, 48: 48}[int(parts[-3])])
                db["Load Case"].append(file.parent.name.replace("_", " "))
                db["Case"].append(" ".join(file.parent.parent.parent.name.split("_")[1:]))
                cases.add(file.parent.parent.parent.name)
                db["Path"].append(file.relative_to(B_PATH).parent)
                break

        # get from file start
        for line in reversed(c):
            if line.startswith("     |  Version : "):
                parts = line.split()

                db["Precision"].append(parts[4])
                db["Version"].append(parts[5])
                db["Parallelization"].append(parts[3])

        # combine
        db["Config"].append(f"{db['Precision'][-1].upper()}P {db['Procs'][-1]}")

        LOG.debug("From %s got %s", file, [(x, db[x][-1]) for x in db.keys()])

    # consolidate
    db = pd.DataFrame(db)
    cases = [" ".join(x.split("_")[1:]) for x in sorted(cases)]
    LOG.info("Got %s", db.shape)
    return db, cases


DB, CASES = get_data()
display(DB)
print(CASES)

In [ ]:
for load_case in DB["Load Case"].unique():
    fig, ax = plt.subplots()
    sns.barplot(data=DB[DB["Load Case"].eq(load_case)], x="Config", y="Time [min]", hue="Case", ax=ax, hue_order=CASES)
    plt.title(load_case)

In [ ]:
for load_case in DB["Load Case"].unique():
    fig, ax = plt.subplots()
    sns.barplot(data=DB[DB["Load Case"].eq(load_case) & DB["Config"].eq("SP 96")], y="Case", x="Time [min]", ax=ax, order=CASES)
    plt.title(load_case)

In [ ]:
display(DB[["Time [min]", "Case"]][DB["Load Case"].eq("Full Frontal") | DB["Load Case"].eq("Moderate Overlap Left")| DB["Load Case"].eq("Moderate Overlap Right")].groupby("Case").median())

In [ ]:
DB[DB["Case"].eq("Base Model")].describe()

In [ ]:
DB["Time [min]"][DB["Case"].eq("Base Model")].plot.hist()

In [ ]:
sns.swarmplot(DB["Time [min]"][DB["Case"].eq("Base Model")])

In [ ]:
fig_width: float = (448.13095 / 72)-0.2
fig_height: float = 0.4 * fig_width
names = {
    "Full Frontal": "Full With Barrier (FWB)",
    "Moderate Overlap Left": "Offset Deformable Barrier\nLeft (ODB-L)",
    "Moderate Overlap Right": "Offset Deformable Barrier\nRight(ODB-R)",
    "Oblique Left": "Oblique Movable Deformable\nBarrier Left (OMDB-L)",
    "Oblique Right": "Oblique Movable Deformable\nBarrier Right (OMDB-R)",
}
fig, ax = plt.subplots(layout="constrained", ncols=2, width_ratios=[1, 0.1])
g = sns.barplot(
    data=DB[DB["Case"].eq("Base Model")].replace(names),
    x="Config",
    y="Time [min]",
    hue="Load Case",
    gap=0.1,
    ax=ax[0],
    palette="colorblind",
    alpha=0.8,
)
ax[0].grid(axis="y")
ax[0].set_ylim([0, 3500])
ax[0].set_yticks(np.arange(0, 3501, 500))
ax[0].set_axisbelow(b=True)
g.legend_.set_title(None)

ax[1].legend(*ax[0].get_legend_handles_labels(), loc="upper center")
ax[1].axis("off")
ax[0].get_legend().remove()
#box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax[0].set_ylabel("Computation Time [min]")
#ax[0].legend(loc="center left", bbox_to_anchor=(1, 0.5))
ax[0].set_xticklabels(
    [
        "Single Precision\n&\n48 Parallel\nProcesses",
        "Single Precision\n&\n96 Parallel\nProcesses",
        "Double Precision\n&\n48 Parallel\nProcesses",
        "Double Precision\n&\n96 Parallel\nProcesses",
    ]
)
ax[0].set_xlabel("")
fig.set_figheight(fig_height)
fig.set_figwidth(fig_width)
fig.savefig(Path("reports") / "Runtime_Base.pdf")